In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohamedhanyyy/chest-ctscan-images")

print("Path to dataset files:", path)

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import TFViTModel
from tensorflow.keras.layers import Input, Dense, Resizing, Rescaling, Permute
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.layers import Input, Dense, Rescaling, Resizing, Dropout, GlobalAveragePooling1D, BatchNormalization, Activation
from tensorflow.keras.models import Model

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')

print("Num GPUs Available: ", len(physical_devices))

for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
data_dir_train = f"{path}/Data/train"
data_dir_val = f"{path}/Data/valid"


train_data = tf.keras.utils.image_dataset_from_directory(
    data_dir_train,
    seed=123,
    image_size=(256,256),
    batch_size=16,
    shuffle=True
)

val_data = tf.keras.utils.image_dataset_from_directory(
    data_dir_val,
    seed=123,
    image_size=(256,256),
    batch_size=16,
    shuffle=False
)

class_names = train_data.class_names
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomZoom(0.3)
])

train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))
train_dataset = train_data.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_data.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
data_dir_test = f"{path}/Data/test"

test_data = tf.keras.utils.image_dataset_from_directory(
    data_dir_test,
    seed=123,
    image_size=(256,256),
    batch_size=16,
    shuffle=False
)

test_dataset = test_data.prefetch(buffer_size=tf.data.AUTOTUNE)